In [6]:
from langchain_openai import AzureChatOpenAI
import requests
from typing import List, Literal
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool

import os
from dotenv import load_dotenv
load_dotenv()

WEATHER_API_KEY = os.environ['WEATHER_API_KEY']
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

llm = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo",
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    openai_api_base=os.getenv("AZURE_OPENAI_API_BASE"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,)

In [7]:
llm.invoke("hi")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_0165350fbb', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-3e01e2b3-5218-451d-962c-e3a7086544f9-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17})

## Defining Langchain Tools

In [8]:
@tool
def get_weather(query: str) -> list:
    """Search weatherapi to get the current weather"""
    endpoint = f"http://api.weatherapi.com/v1/current.json?key={WEATHER_API_KEY}&q={query}"
    response = requests.get(endpoint)
    data = response.json()

    if data.get("location"):
        return data
    else:
        return "Weather Data Not Found"

@tool
def search_web(query: str) -> list:
    """Search the web for a query"""
    tavily_search = TavilySearchResults(api_key=TAVILY_API_KEY, max_results=2, search_depth='advanced', max_tokens=1000)
    results = tavily_search.invoke(query)
    return results

To make these tools accessible to the LLM, we can bind them to the model as shown below. In this example, we’re using the Together API to utilize the Llama 3 model. If you have an OpenAI key, you can proceed with that by specifying your key instead.

In [9]:
tools = [search_web, get_weather]
llm_with_tools = llm.bind_tools(tools)

In [10]:
llm_with_tools

RunnableBinding(bound=AzureChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000020D4A664710>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000020D4A51CE90>, root_client=<openai.lib.azure.AzureOpenAI object at 0x0000020D4678CEF0>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x0000020D4A664740>, temperature=0.0, deployment_name='gpt-35-turbo', openai_api_version='2023-07-01-preview'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'search_web', 'description': 'Search the web for a query', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string'}}, 'required': ['query']}}}, {'type': 'function', 'function': {'name': 'get_weather', 'description': 'Search weatherapi to get the current weather', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string'}}, 'required': ['query']}}}]})

Now, lets invoke the LLM with a prompt to see the results:

In [11]:
prompt = """
    Given only the tools at your disposal, mention tool calls for the following tasks:
    Do not change the query given for any search tasks
        1. What is the current weather in Trivandrum today
        2. Can you tell me about Kerala
        3. Why is the sky blue?
    """

results = llm_with_tools.invoke(prompt)

print(results.tool_calls)

query = "What is the current weather in Trivandrum today"
response = llm.invoke(query)
print(response.content)

[{'name': 'get_weather', 'args': {'query': 'Trivandrum'}, 'id': 'call_DPGgcVb8ZoPVZkPHnH4Fn0bd', 'type': 'tool_call'}, {'name': 'search_web', 'args': {'query': 'Kerala'}, 'id': 'call_DIlsgdfivziQOfjkjDzbSXS2', 'type': 'tool_call'}, {'name': 'search_web', 'args': {'query': 'Why is the sky blue?'}, 'id': 'call_DJUm4Yb5N58vyZRGrMm9KRk3', 'type': 'tool_call'}]
I am an AI and do not have real-time data. I recommend checking a reliable weather website or app for the most up-to-date weather information in Trivandrum.


## 2. Using pre-built agent

#### LangGraph offers a pre-built React(Reason and Act) agent, designed to streamline decision-making and action execution.

Let’s explore how it works.

In [12]:
from langgraph.prebuilt import create_react_agent

# system prompt is used to inform the tools available to when to use each
system_prompt = """Act as a helpful assistant.
    Use the tools at your disposal to perform tasks as needed.
        - get_weather: whenever user asks get the weather of a place.
        - search_web: whenever user asks for information on current events or if you don't know the answer.
    Use the tools only if you don't know the answer.
    """

agent = create_react_agent(model=llm, tools=tools, state_modifier=system_prompt)

# Let’s query the agent to see the result.
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [("user", "What is the current weather in Trivandrum today")]}

print_stream(agent.stream(inputs, stream_mode="values"))

c:\Users\sayantghosh\AppData\Local\anaconda3\envs\genai\Lib\site-packages\pydantic\_internal\_generate_schema.py:898: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `BaseMessage` to V2.
  warn(


================================ Human Message =================================

What is the current weather in Trivandrum today
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_SgsNT0JDBdlbVRB0poneGAf4)
 Call ID: call_SgsNT0JDBdlbVRB0poneGAf4
  Args:
    query: Trivandrum
================================= Tool Message =================================
Name: get_weather

{"location": {"name": "Trivandrum", "region": "Kerala", "country": "India", "lat": 8.5069, "lon": 76.9569, "tz_id": "Asia/Kolkata", "localtime_epoch": 1748937120, "localtime": "2025-06-03 13:22"}, "current": {"last_updated_epoch": 1748936700, "last_updated": "2025-06-03 13:15", "temp_c": 30.2, "temp_f": 86.4, "is_day": 1, "condition": {"text": "Partly cloudy", "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png", "code": 1003}, "wind_mph": 12.3, "wind_kph": 19.8, "wind_degree": 295, "wind_dir": "WNW", "pressure_mb": 1009.0, "pressure_in": 29.8, "preci

In [16]:
# import the required methods
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState, START, END
from PIL import Image

# define a tool_node with the available tools
tools = [search_web, get_weather]
tool_node = ToolNode(tools)

# define functions to call the LLM or the tools
def call_model(state: MessagesState):
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def call_tools(state: MessagesState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

# initialize the workflow from StateGraph
workflow = StateGraph(MessagesState)

# add a node named LLM, with call_model function. This node uses an LLM to make decisions based on the input given
workflow.add_node("LLM", call_model)

# Our workflow starts with the LLM node
workflow.add_edge(START, "LLM")

# Add a tools node
workflow.add_node("tools", tool_node)

# Add a conditional edge from LLM to call_tools function. It can go tools node or end depending on the output of the LLM. 
workflow.add_conditional_edges("LLM", call_tools)

# tools node sends the information back to the LLM
workflow.add_edge("tools", "LLM")

agent = workflow.compile()

In [17]:

for chunk in agent.stream(
    {"messages": [("user", "Will it rain in Trivandrum today?")]},
    stream_mode="values",):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

Will it rain in Trivandrum today?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_my5XSAHGN3XBT8grIUjWZ1oc)
 Call ID: call_my5XSAHGN3XBT8grIUjWZ1oc
  Args:
    query: Trivandrum
================================= Tool Message =================================
Name: get_weather

{"location": {"name": "Trivandrum", "region": "Kerala", "country": "India", "lat": 8.5069, "lon": 76.9569, "tz_id": "Asia/Kolkata", "localtime_epoch": 1748937505, "localtime": "2025-06-03 13:28"}, "current": {"last_updated_epoch": 1748936700, "last_updated": "2025-06-03 13:15", "temp_c": 30.2, "temp_f": 86.4, "is_day": 1, "condition": {"text": "Partly cloudy", "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png", "code": 1003}, "wind_mph": 12.3, "wind_kph": 19.8, "wind_degree": 295, "wind_dir": "WNW", "pressure_mb": 1009.0, "pressure_in": 29.8, "precip_mm": 0.01, "